In [1]:
import intake
import sys
import logging
from collections.abc import Iterable
import dask

cat = intake.open_catalog("../main.yaml")

In [2]:
def tree(cat, level=0, stream=sys.stdout):
    prefix = " " * (3 * level)
    try:
        for child in list(cat):
            parameters = [
                p["name"] for p in cat[child].describe().get("user_parameters", [])
            ]
            if len(parameters) > 0:
                parameter_str = " (" + ", ".join(parameters) + ")"
            else:
                parameter_str = ""
            mds = get_metadata_string(cat[child])
            dask_test = test_dask(cat[child])
            print(prefix + str(child) + parameter_str + mds + dask_test, file=stream)
            tree(cat[child], level + 1)

    except TypeError as e:  # we are probably trying to iterate a zarr file or so.
        if "object is not iterable" not in str(e):
            raise e


def get_metadata_string(entry):
    if not isinstance(entry, Iterable):
        md = entry.metadata
        try:
            mds = f"\tP: {md['project']}, E: {md['experiment_id']}, Model: {md['source_id']}, ID: {md['simulation_id']}"
        except KeyError as e:
            logging.critical(f"Cannot process {entry}. Metadata key {str(e)} missing")
            raise e
        return mds
    else:
        return ""
    
def test_dask(entry):
    if not isinstance(entry, Iterable):
        try: 
            with dask.config.set(**{'array.slicing.split_large_chunks': True}):
                vars = list(entry.to_dask().variables.keys())
        except Exception as e:
            logging.critical(f"Cannot convert {entry} to dask.")
            raise e

        return f", {len(vars)} Variables"
    else:
        return ""

In [3]:
tree(cat)

ngc4008 (time, zoom)	P: nextGEMS, E: nextgems_prefinal, Model: ICON-ESM, ID: ngc4008, 108 Variables
ngc4006 (time, zoom)	P: nextGEMS, E: nextgems_prefinal, Model: ICON-ESM, ID: ngc4006, 106 Variables
ngc4005 (time, zoom)	P: nextGEMS, E: nextgems_prefinal, Model: ICON-ESM, ID: ngc4005, 112 Variables
ngc3028 (time, zoom)	P: nextGEMS, E: nextgems_cycle3, Model: ICON-ESM, ID: ngc3028, 102 Variables
ngc3028_bc_land	P: nextGEMS, E: nextgems_cycle3, Model: ICON-ESM, ID: ngc3028, 43 Variables
ngc3026_WILL_BE_DELETED (time, zoom)	P: nextGEMS, E: nextgems_cycle3, Model: ICON-ESM, ID: ngc3026, 96 Variables
HAMOCC
   HEALPix (zoom)	P: nextGEMS, E: nextgems_cycle3, Model: ICON-ESM, ID: cycle3_HAMOCC, 94 Variables
   OneDegree	P: nextGEMS, E: nextgems_cycle3, Model: ICON-ESM, ID: cycle3_HAMOCC, 71 Variables
slow
   ngc3028 (time, zoom)	P: nextGEMS, E: nextgems_cycle3, Model: ICON-ESM, ID: ngc3028, 102 Variables
   HAMOCC
      HEALPix (zoom)	P: nextGEMS, E: nextgems_cycle3, Model: ICON-ESM, ID: cycl